In [2]:
%matplotlib

Using matplotlib backend: Qt5Agg


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import math
import cmath
import numpy.fft as fft
import ipywidgets as widgets
from IPython.display import display

1: Define a Potential, for example infinitely deep well <br>
2: Find/Select (?) a starting wave function $\Psi_0 = \Psi(x, t=0)$ <br>
3: Time-evolution in first order: 


$\Psi(x, t+\Delta t) = e^{-\frac{i \Delta t}{2 \hbar} V(x)} \cdot \mathscr{F}^{-1}\left[ e^{-\frac{i \hbar \Delta t p^2}{2 m}} \cdot \mathscr{F}\left[ e^{-\frac{i \Delta t}{2 \hbar} V(x)} \cdot \Psi(x, t) \right] \right]$

In [5]:
# constants
h_bar = 1.054e-34       # hbar obviously
amu = 1.660538921e-27   # atomic mass unit
m = 87*amu              # atomic mass (of rubidium)
a_s = 5.8e-9            # scattering length
N = 1000                # number of atoms
wr = 100 * 2 * np.pi    # trapping frequency (TODO: read about this)
wx = 40 * 2 * np.pi     # trapping frequency (TODO: read about this)

In [6]:
M = 200 
Nx = 2 * M + 1               # length of x array
dx = 2e-7                    # coordinate step
x = np.arange(-M, M+1, 1)*dx # coordinate range (array)
dk = np.pi/(M*dx)            # k space step
k = np.arange(-M, M+1, 1)*dk # k space range (array)
dt = 10e-8                   # time step
Nt = 300000                  # number of time steps

In [11]:
lr = np.sqrt(h_bar/(m*wr))     # harmonic oscillator length
lx = np.sqrt(h_bar/(m+wx))     # harmonic oscillator length
g1d = 2*h_bar**2*a_s/(m*lr**2) # 1D interaction coefficient

V = 0.5*m*wx**2*np.power(x, 2)/h_bar                                      # trapping potential (array)
psi_0 = np.sqrt(N/lx)*(1/np.pi)**(1/4)*np.exp(-np.power(x, 2)/(2*lx**2))  # starting wave function Psi_0 (array)

In [12]:
plt.plot(psi_0)
plt.show()

In [14]:
Nframe = 500  # number of frames to capture
t = 0         # t is zero at the start

#psi_0, mu = get_ground_state(psi_0, dt, g1d, x, k, m, V)

psi = psi_0 # since get_ground_state doesnt work yet we are looking at our non-gound-state wave function

spacetime = [] # no idea why this is called space time but it saves the frames

for itime in range(1, Nt+1):                                            # iterate over the time
    psi=psi*np.exp(-0.5*1j*dt*(V+(g1d/h_bar)*np.power(np.abs(psi), 2))) # all these steps basically implement the individual multiplications in the formula given above
    psi_k=fft.fftshift(fft.fft(psi))/Nx                                 # doing the fft and doning a fft-shift
    psi_k=psi_k*np.exp(-0.5*1j*dt*(h_bar/m)*np.power(k, 2))             # second multiplication
    psi=fft.ifft(fft.ifftshift(psi_k))*Nx                               # inverse fft-shift and fft
    psi=psi*np.exp(-0.5*1j*dt*(V+(g1d/h_bar)*np.power(np.abs(psi), 2))) # third multiplication
    if itime%(Nt/Nframe) == 0:
        spacetime.append(np.abs(np.power(psi, 2)))                      # condition to check wether a frame has to be saved or not
    t += dt                                                             # time step

In [17]:
plt.ion()
graph = plt.plot(np.abs(np.power(psi_0, 2)))[0]

for a in spacetime:
    graph.set_ydata(a)
    plt.xlim(150, 250)
    plt.draw()
    plt.pause(0.01)

#### Ende der Physik
Alles ab hier funktioniert noch nicht so richtig <br>
Die Funktion get_ground_state soll mittels imaginary time method den Grundzutand finden

In [ ]:
def get_ground_state(psi, dt, g1d, x, k, m, V):
    h_bar = 1.054e-34
    dx = x[1]-x[0]
    dk = 2*np.pi/(x[-1]-x[0])
    N = dx * np.power( psi / np.linalg.norm(psi, 2), 2)
    Nx = len(x)
    psi_mid_old = psi[Nx//2]
    j = 1
    mu = 1
    mu_old = 1
    mu_error = 1
    
    while mu_error > 1e-12:
        print("== next iteration ==\n")
        #print("Psi == 0 at %d positions" % len(np.where(psi == 0)[0]))
        psi=psi*np.exp(-0.5*dt*(V+(g1d/h_bar)*np.power(np.abs(psi), 2)))
        #print("Psi == 0 at %d positions" % len(np.where(psi == 0)[0]))
        #print("Psi NaN", np.where(np.isnan(psi)))
        psi_k=fft.fftshift(fft.fft(psi))/Nx
        #print("Psi_k NaN", np.where(np.isnan(psi_k)))
        #print("Psi == 0 at %d positions" % len(np.where(psi == 0)[0]))
        psi_k=psi_k*np.exp(-0.5*dt*(h_bar/m)*np.power(k, 2))
        #print("Psi_k == 0", np.where(psi_k == 0))
        psi=fft.ifft(fft.ifftshift(psi_k))*Nx
        #print("Psi == 0 at %d positions" % len(np.where(psi == 0)[0]))
        psi=psi*np.exp(-0.5*dt*(V+(g1d/h_bar)*np.power(np.abs(psi), 2))) # first order
        #print("Psi == 0 at %d positions" % len(np.where(psi == 0)[0]))
        psi_mid = psi[Nx//2]

        #print("psi_old = {}, psi_new = {}".format(psi_mid_old, psi_mid))
        mu = math.log(abs(psi_mid_old/psi_mid))/dt
        mu_error = abs(mu-mu_old)/mu
        #print("mu = {}, mu_error = {}".format(mu, mu_error))
        
        #print("\nPsi NaN before weird step", np.where(np.isnan(psi)))
        #print("Substituting any zeros")
        psi[np.where(psi == 0)[0]] = 1e-30
        
        #denum = np.sqrt(dx * np.power( psi / np.linalg.norm(psi, 2), 2))
        #print("Denum == 0", np.where(denum == 0))
        print("Psi == 0", np.where(psi == 0))
        
        
        psi = psi*np.sqrt(N)/np.sqrt(dx * np.power( psi / np.linalg.norm(psi, 2), 2))
        #print("Psi NaN after weird step", np.where(np.isnan(psi)))
        if j%5000 == 0:
            print("MU_ERROR")
        if j > 1e8:
            print("no solution found")
            return psi, mu
        psi_mid_old = psi[Nx//2]
        mu_old = mu
        j += 1
    print(j-1, "iterations needed")
    return psi, mu  

In [ ]:
# %matplotlib inline

In [ ]:
psi_0 = np.sqrt(N/lx)*(1/np.pi)**(1/4)*np.exp(-np.power(x, 2)/(2*lx**2+1e-13))
#psi_0 = psi_0 / np.linalg.norm(psi_0)
plt.plot(psi_0)

psi_0, mu = get_ground_state(psi_0, dt, g1d, x, k, m, V)
plt.plot(np.abs(psi_0))
plt.show()

In [ ]:
psi_0 = np.exp(-np.power(x, 2)/(1e-9))
print(len(np.where(psi_0 != 0)[0]))
plt.plot(psi_0)
plt.plot(fft.fft(psi_0))
plt.show()